In [12]:
import Rust_Machine_Learning
import plotly.express as px
import pandas as pd

In [13]:
# Get the csv data
# df = pd.read_csv("../data/iris.data")
# df = pd.read_csv("../data/kmeans-dataset.csv")
# df = pd.read_csv("../data/winequality-red.csv")
df = pd.DataFrame({
  "Dosage": [0.0,2.0,4.0,8.0,11.0,13.0,15.0,18.0,20.0,22.0,25.0,26.0,27.0,28.0,29.0,31.0,34.0,36.0,37.0],
  "Effectiveness": [0.0,0.0,0.0,0.0,5.0,18.0,100.0,100.0,100.0,100.0,60.0,58.0,56.0,52.0,48.0,15.0,0.0,0.0,0.0]
})
df  


,Dosage,Effectiveness
0,0.0,0.0
1,2.0,0.0
2,4.0,0.0
3,8.0,0.0
4,11.0,5.0
5,13.0,18.0
6,15.0,100.0
7,18.0,100.0
8,20.0,100.0
9,22.0,100.0


In [14]:
# Ensure categoriical data is one-hot encoded
# columns is list of ints
def one_hot_encode(df, columns):
    encoded = df.copy()
    feature_to_cols = dict()
    for col in columns:
        # Get the unique values
        unique_vals = encoded.iloc[:, col].unique()

        # Start col
        start_col = encoded.shape[1]-1

        # Now for each unique value add a col to df
        for val in unique_vals:
            col_encoded = encoded.iloc[:, col] == val
            col_encoded = [(1.0 if label == True else 0.0) for label in col_encoded]
            encoded["is_" + encoded.columns[col] + "_" + str(val)] = col_encoded
        
        end_col = encoded.shape[1]-1
        feature_to_cols[col] = list(range(start_col, end_col))

    encoded.drop(encoded.columns[columns], axis=1, inplace=True)
    return (encoded, feature_to_cols)
to_encode = []
encoded, feature_to_encoded_cols = one_hot_encode(df, to_encode)
encoded


IndexError: single positional indexer is out-of-bounds

In [ ]:
# Split the data into train and test
def split_train_test(df, percent_train, randomize = True):
  #Randomize the rows
  if randomize:
    df = df.sample(frac=1.0)

  #Create train and test data
  df_size = df.shape[0]
  train_size = df_size * percent_train

  train_data = df.iloc[:int(train_size),:]
  test_data = df.iloc[int(train_size):,:]
  return (train_data, test_data)

In [ ]:
def remove_encoded_category_data_and_label(encoded, feature_to_encoded_cols, label_col):
  cols_to_remove = []
  did_remove_label_col = False
  for k,v in feature_to_encoded_cols.items():
    if label_col in v:
      encoded.drop(encoded.columns[v], axis=1, inplace=True)
      did_remove_label_col = True
  if not did_remove_label_col:
    encoded.drop(encoded.columns[label_col], axis=1, inplace=True)
    

In [ ]:
# Now setup the train/test data
# We will train it to predict y based on x and cluster
label_col = 1

train_data, test_data = split_train_test(encoded, 1.0, False)

# Feature Data
train_data_features = train_data.copy()
remove_encoded_category_data_and_label(train_data_features, feature_to_encoded_cols, label_col)
train_data_features_list = train_data_features.values.tolist()

# Label Data
train_data_labels = train_data[train_data.columns[label_col]].values.tolist()
train_data_features

,x,is_cluster_0,is_cluster_1,is_cluster_2,is_cluster_3,is_cluster_4,is_cluster_5,is_cluster_8,is_cluster_7,is_cluster_9
0,1.000070,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.997360,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.134799,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2.475850,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-3.058700,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
2985,98.255900,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2986,96.500700,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2987,96.885200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2988,96.834000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
# Now train the model
trained_model = Rust_Machine_Learning.RegressionTree(train_data_features_list, train_data_labels)
trained_model.print()

 7 0.5
   8 0.5
     0 65.84075
       1 0.5
         0 8.539805000000001
           0 8.034089999999999
             0 5.55422
               0 4.176525
                 0 4.118625
                   0 -10.048045
                     -5.77052
                     0 -2.464775
                       0 -3.00048
                         0 -3.2977350000000003
                           0 -5.68542
                             0 -6.016439999999999
                               0 -9.656715
                                 3.61729
                                 0 -9.56908
                                   -4.2444
                                   0 -7.4415700000000005
                                     0 -8.019459999999999
                                       1.7032281666666664
                                       -2.8350553333333335
                                     0 -6.595000000000001
                                       3.26278
                                       0.24084

In [ ]:
# Test the model